In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pd.options.plotting.backend = 'plotly'


For the purpose of this code, Austin Reaves will be the subject. First, we must establish of point of comparison of least year's stats as to determine if he has truly regressed. All data comes from basketball-reference.com as they prepare all game and season logs as csv form.

In [3]:
reaves_22_23_path = Path('data') / 'reaves_22_23_game_logs.csv'
ar22_23 = pd.read_csv(reaves_22_23_path)
ar22_23.head()

,G,Date,Opp,Unnamed: 3,GS,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,10/18/2022,GSW,L (-14),0,16:00,1,3,0.333,0,...,0,0,1,0,0,0,1,3,1.6,-6
2,2.0,10/20/2022,LAC,L (-6),0,21:55,1,2,0.5,0,...,2,4,2,0,0,0,5,4,4.4,0
3,3.0,10/23/2022,POR,L (-2),0,18:58,2,5,0.4,2,...,0,0,1,1,0,2,1,7,3.6,-6
4,4.0,10/26/2022,DEN,L (-11),1,26:29:00,3,5,0.6,2,...,2,2,1,0,0,1,0,8,6,-23


Fortunately, the data is fairly clean. But I will not consider any games he did not play. So I will drop games that he did not play.

In [4]:
def clean_player_data(game_logs):
    game_logs = game_logs.rename(columns={"Unnamed: 3": "Result"})
    game_logs = game_logs.replace("Inac", np.NaN)
    game_logs = game_logs.replace("Did", np.NaN)
    game_logs = game_logs.dropna(subset=['MP'])
    game_logs["G"] = game_logs['G'].apply(int)
    game_logs = game_logs.fillna(0)
    game_logs["GS"] = game_logs['GS'].apply(int)
    game_logs.loc[:, "FG": "+/-"] = game_logs.loc[:, "FG": "+/-"].map(float)
    return game_logs

In [5]:
cleaned_ar = clean_player_data(ar22_23)
cleaned_ar.tail()

,G,Date,Opp,Result,GS,MP,FG,FGA,FG%,3P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
78,60,4/2/2023,HOU,W (+25),1,32:44:00,4.0,7.0,0.571,1.0,...,1.0,1.0,8.0,0.0,0.0,2.0,2.0,18.0,17.4,15.0
79,61,4/4/2023,UTA,W (+2),1,41:32:00,8.0,13.0,0.615,4.0,...,3.0,3.0,6.0,0.0,0.0,2.0,1.0,28.0,24.8,10.0
80,62,4/5/2023,LAC,L (-7),1,30:41:00,6.0,11.0,0.545,3.0,...,4.0,4.0,2.0,0.0,0.0,2.0,5.0,20.0,13.3,9.0
81,63,4/7/2023,PHO,W (+14),1,31:15:00,9.0,13.0,0.692,1.0,...,0.0,0.0,5.0,1.0,0.0,2.0,1.0,22.0,17.8,18.0
82,64,4/9/2023,UTA,W (+11),1,33:59:00,3.0,5.0,0.6,3.0,...,4.0,4.0,6.0,0.0,0.0,0.0,2.0,12.0,14.3,20.0


As Reaves only played 64 games in a season, the relevant data is now saved in the cleaned dataframe. NaN values resulting from values not being calculated are set as zero. Furthermore, we will consider all numeric, continuous variables as floats. Whenever we calculate averages, they tend to be floats anyways.

In [6]:
def mean_calc(stat):
    return stat.mean()

In [7]:
def sum_calc(stat):
    return stat.sum()

In [9]:
ar_average_points = mean_calc(cleaned_ar['PTS'])
ar_average_points

13.015625

In [ ]:
ar_total_points = sum_calc(cleaned_ar['PTS'])
ar_total_points

833.0

Now we can calcualte relevant values for comparison. It is necessary to make the distinction between average and totals as certain metrics such as +/- is a cumulative stat. 

In [15]:
league_totals_22_23_path = Path('data') / 'league_totals_22_23.csv'
totals_22_23 = pd.read_csv(league_totals_22_23_path, encoding = 'latin-1')
totals_22_23.head()

,Rk,Player,Pos,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Player-additional
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999
1,1.0,Precious Achiuwa,C,TOR,55.0,12.0,1140.0,196.0,404.0,0.485,...,100.0,228.0,328.0,50.0,31.0,30.0,59.0,102.0,508.0,achiupr01
2,2.0,Steven Adams,C,MEM,42.0,42.0,1133.0,157.0,263.0,0.597,...,214.0,271.0,485.0,97.0,36.0,46.0,79.0,98.0,361.0,adamsst01
3,3.0,Bam Adebayo,C,MIA,75.0,75.0,2598.0,602.0,1114.0,0.540,...,184.0,504.0,688.0,240.0,88.0,61.0,187.0,208.0,1529.0,adebaba01
4,4.0,Ochai Agbaji,SG,UTA,59.0,22.0,1209.0,165.0,386.0,0.427,...,43.0,78.0,121.0,67.0,16.0,15.0,41.0,99.0,467.0,agbajoc01


Ok, so now we want to load in league averages and totals to get a true sense of where Reaves was at last year.

In [26]:
def clean_league_data(league_stats):
    league_stats = league_stats.drop(columns=['Player-additional'])
    league_stats = league_stats.dropna(how='all')
    league_stats['Rk'] = league_stats['Rk'].apply(int)
    league_stats.loc[:, "G":'PTS'] = league_stats.loc[:, "G":'PTS'].map(float)
    return league_stats

In [28]:
clean_totals_22_23 = clean_league_data(totals_22_23)
clean_totals_22_23.head()

,Rk,Player,Pos,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,Precious Achiuwa,C,TOR,55.0,12.0,1140.0,196.0,404.0,0.485,...,0.702,100.0,228.0,328.0,50.0,31.0,30.0,59.0,102.0,508.0
2,2,Steven Adams,C,MEM,42.0,42.0,1133.0,157.0,263.0,0.597,...,0.364,214.0,271.0,485.0,97.0,36.0,46.0,79.0,98.0,361.0
3,3,Bam Adebayo,C,MIA,75.0,75.0,2598.0,602.0,1114.0,0.540,...,0.806,184.0,504.0,688.0,240.0,88.0,61.0,187.0,208.0,1529.0
4,4,Ochai Agbaji,SG,UTA,59.0,22.0,1209.0,165.0,386.0,0.427,...,0.812,43.0,78.0,121.0,67.0,16.0,15.0,41.0,99.0,467.0
5,5,Santi Aldama,PF,MEM,77.0,20.0,1682.0,247.0,525.0,0.470,...,0.750,85.0,286.0,371.0,97.0,45.0,48.0,60.0,143.0,696.0


It turns out there are over 500 players that played at least 1 game in the NBA last season. But many of those do not serve in comparison to Reaves. For example, Thanasis Antetokounmpo.

In [31]:
clean_totals_22_23[clean_totals_22_23['Player'] == "Thanasis Antetokounmpo"]

,Rk,Player,Pos,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
14,12,Thanasis Antetokounmpo,PF,MIL,37.0,0.0,206.0,20.0,46.0,0.435,...,0.5,15.0,29.0,44.0,15.0,3.0,3.0,12.0,22.0,51.0


In [32]:
league_average_mp = mean_calc(clean_totals_22_23['MP'])
league_average_mp

984.421207658321

What we will do is remove players who have under the league average for minutes played.

In [33]:
clean_totals_22_23 = clean_totals_22_23[clean_totals_22_23['MP'] >= league_average_mp]
clean_totals_22_23

,Rk,Player,Pos,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
1,1,Precious Achiuwa,C,TOR,55.0,12.0,1140.0,196.0,404.0,0.485,...,0.702,100.0,228.0,328.0,50.0,31.0,30.0,59.0,102.0,508.0
2,2,Steven Adams,C,MEM,42.0,42.0,1133.0,157.0,263.0,0.597,...,0.364,214.0,271.0,485.0,97.0,36.0,46.0,79.0,98.0,361.0
3,3,Bam Adebayo,C,MIA,75.0,75.0,2598.0,602.0,1114.0,0.540,...,0.806,184.0,504.0,688.0,240.0,88.0,61.0,187.0,208.0,1529.0
4,4,Ochai Agbaji,SG,UTA,59.0,22.0,1209.0,165.0,386.0,0.427,...,0.812,43.0,78.0,121.0,67.0,16.0,15.0,41.0,99.0,467.0
5,5,Santi Aldama,PF,MEM,77.0,20.0,1682.0,247.0,525.0,0.470,...,0.750,85.0,286.0,371.0,97.0,45.0,48.0,60.0,143.0,696.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,523,Patrick Williams,PF,CHI,82.0,65.0,2323.0,314.0,677.0,0.464,...,0.857,78.0,249.0,327.0,100.0,72.0,70.0,101.0,147.0,833.0
671,531,Christian Wood,C,DAL,67.0,17.0,1738.0,396.0,769.0,0.515,...,0.772,87.0,404.0,491.0,121.0,30.0,72.0,121.0,169.0,1114.0
672,532,Delon Wright,PG,WAS,50.0,14.0,1221.0,138.0,291.0,0.474,...,0.867,58.0,122.0,180.0,194.0,92.0,17.0,44.0,59.0,369.0
676,536,Trae Young,PG,ATL,73.0,73.0,2541.0,597.0,1390.0,0.429,...,0.886,56.0,161.0,217.0,741.0,80.0,9.0,300.0,104.0,1914.0


Now, this cuts down over 50 percent of players. Although starters appear to play more than 2000 minutes, they can serve as comparators. For example, last years sixth man of the year, Malcolm Brogdon.

In [35]:
clean_totals_22_23[clean_totals_22_23['Player'] == "Malcolm Brogdon"]

,Rk,Player,Pos,Tm,G,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
82,60,Malcolm Brogdon,PG,BOS,67.0,0.0,1744.0,354.0,732.0,0.484,...,0.87,42.0,238.0,280.0,248.0,45.0,18.0,98.0,109.0,1000.0


Now that we have the data for the entire league, we can start making comparisons. For the sake of simplicity, we will take 8 categories as a player's output.
- Rebouding
- Scoring
- Efficiency (True Shooting Percentage)
- 3 Pointers
- Playmaking (Assists)
- Ball Handling (Assist / Turnover Ratio)
- Open Court Defense (Steals)
- At Rim Defense (Blocks)

Albeit, these metrics are simplications of what is actually happening on the court. Coachs' strategical play can affect what players are doing on the court. I.e. Just because a player may not have high steals may not be imply he is a bad defender. Nevertheless, someone with high amount of steals still highlights good open court defense. 